In [ ]:
import pandas as pd
import numpy as np
import re

In [3]:
# Mostrar hasta 100 filas en la salida
pd.set_option('display.max_rows', 100)

# Mostrar hasta 100 columnas (opcional)
pd.set_option('display.max_columns', 100)

# Mostrar hasta 200 caracteres por columna
pd.set_option('display.max_colwidth', 200)

In [4]:
# Define la ruta completa de tu archivo Parquet
ruta_archivo_parquet = r"C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\datos_todos_los_pdf.parquet"


# Lee el archivo Parquet en un DataFrame
df = pd.read_parquet(ruta_archivo_parquet)

In [7]:
# Filtrar Filas con vencimientos de opciones y tasa libre de riesgo
df_filtrado = df[df['Texto'].str.startswith('GFG')]

In [27]:
# Ordenar por fecha
df_ordenado = df_filtrado.sort_values(by='Fecha')

# Eliminar Duplicados
df_sin_duplicados = df_ordenado.drop_duplicates()

# Resetear Indice
df_sin_duplicados = df_sin_duplicados.reset_index(drop=True)

# Split de la columna Texto usando el guion
df_separado = df_sin_duplicados['Texto'].str.split(' ', expand=True)

# Concatenar con el DF previo para mantener la columna fecha
df_final = pd.concat([df_sin_duplicados[['Fecha']].reset_index(drop=True), df_separado], axis=1)

In [28]:
# Nuevos nombres de columna:
df_final.columns = ['Fecha','Serie', 'Ultimo_Precio', 'Variacion_%','Min', 'Max','Lotes','Volumen_$','Ult_Precio/Cotiz_sub_%','Prima_Teorica','Delta','Efec_Palanca','Volat_Implicita_%','Delta_Implic']

In [41]:
# Quitar " %" de columna:
cols = ['Variacion_%', 'Ult_Precio/Cotiz_sub_%', 'Volat_Implicita_%','Efec_Palanca']

for col in cols:
    df_final[col] = df_final[col].astype(str).str.replace('%', '', regex=False)

In [ ]:
cols = ['Variacion_%', 'Ult_Precio/Cotiz_sub_%', 'Volat_Implicita_%','Efec_Palanca']

for col in cols:
    df_final[col] = df_final[col].replace('-', np.nan)
    # Intentamos convertir, pero sin eliminar comas
    df_final[col] = pd.to_numeric(df_final[col], errors='coerce')


for col in cols:
    # Intentamos convertir a float, errores quedan como NaN
    converted = pd.to_numeric(
        df_final[col].astype(str).str.replace('%', '', regex=False).str.replace(',', '', regex=False),
        errors='coerce'
    )
    # Filtramos los valores que NO se convirtieron (quedaron NaN) pero que no son NaN originales
    mask = converted.isna() & (~df_final[col].isna())
    print(f"Valores no convertibles en columna '{col}':")
    print(df_final.loc[mask, col])
    print('-'*40)


Valores no convertibles en columna 'Variacion_%':
Series([], Name: Variacion_%, dtype: float64)
----------------------------------------
Valores no convertibles en columna 'Ult_Precio/Cotiz_sub_%':
Series([], Name: Ult_Precio/Cotiz_sub_%, dtype: float64)
----------------------------------------
Valores no convertibles en columna 'Volat_Implicita_%':
Series([], Name: Volat_Implicita_%, dtype: float64)
----------------------------------------
Valores no convertibles en columna 'Efec_Palanca':
Series([], Name: Efec_Palanca, dtype: float64)
----------------------------------------


In [44]:
cols = ['Variacion_%', 'Ult_Precio/Cotiz_sub_%', 'Volat_Implicita_%','Efec_Palanca']

for col in cols:
    # Intentamos convertir quitando '-', pero manteniendo comas
    converted = pd.to_numeric(
        df_final[col].astype(str).replace('-', np.nan),
        errors='coerce'
    )
    # Valores que quedaron NaN y no eran NaN originales (es decir, no convertibles)
    mask = converted.isna() & (~df_final[col].isna())
    print(f"Valores no convertibles en columna '{col}':")
    print(df_final.loc[mask, col])
    print('-'*40)


Valores no convertibles en columna 'Variacion_%':
Series([], Name: Variacion_%, dtype: float64)
----------------------------------------
Valores no convertibles en columna 'Ult_Precio/Cotiz_sub_%':
Series([], Name: Ult_Precio/Cotiz_sub_%, dtype: float64)
----------------------------------------
Valores no convertibles en columna 'Volat_Implicita_%':
Series([], Name: Volat_Implicita_%, dtype: float64)
----------------------------------------
Valores no convertibles en columna 'Efec_Palanca':
Series([], Name: Efec_Palanca, dtype: float64)
----------------------------------------


In [47]:
# Convertir en tipo Fecha
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'], errors='coerce')

# Desde la tercera columna en adelante convertir en float
for col in df_final.columns[2:]:  
    if df_final[col].dtype == 'object':
        df_final[col] = df_final[col].str.replace(',', '', regex=False)
        df_final[col] = df_final[col].astype(float)


In [48]:
df_final

,Fecha,Serie,Ultimo_Precio,Variacion_%,Min,Max,Lotes,Volumen_$,Ult_Precio/Cotiz_sub_%,Prima_Teorica,Delta,Efec_Palanca,Volat_Implicita_%,Delta_Implic
0,2010-01-04,GFGV2.05EN,0.020,-66.67,0.020,0.020,1000.0,2000.0,0.92,0.008,-0.128,33.50,45.65,-0.199
1,2010-01-04,GFGC2.25EN,0.039,11.43,0.039,0.039,200.0,780.0,1.79,0.026,0.321,27.21,42.78,0.364
2,2010-01-04,GFGC1.75FE,0.437,1.63,0.437,0.437,30.0,1311.0,20.05,0.453,0.978,4.70,0.00,1.000
3,2010-01-04,GFGC1.85FE,0.370,8.82,0.370,0.370,30.0,1110.0,16.97,0.360,0.938,5.69,42.05,0.894
4,2010-01-04,GFGC1.95FE,0.290,52.63,0.290,0.300,665.0,19450.0,13.30,0.272,0.864,6.91,42.54,0.813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108268,2024-04-23,GFGC4050JU,51.874,-16.47,50.000,67.000,2849.0,17442803.0,1.62,120.850,0.309,8.21,35.89,0.210
108269,2024-04-23,GFGC4200JU,35.189,-20.38,32.990,48.990,6278.0,26059354.0,1.10,92.060,0.251,8.76,36.70,0.152
108270,2024-04-23,GFGC2800AG,800.000,14.29,800.000,800.000,25.0,2000000.0,24.92,975.826,0.904,2.97,0.00,1.000
108271,2024-04-23,GFGC2900JU,584.000,-10.34,580.000,694.000,177.0,10984764.0,18.19,651.873,0.860,4.24,0.00,1.000


In [133]:
def separar_celda(valor):
    valor = str(valor)
    cuarto_char = valor[3] if len(valor) > 3 else ''
    
    if cuarto_char == 'C': # Opcion de Compra o Call
        opcion = 1
    elif cuarto_char == 'V':# Opcion de Venta o Put
        opcion = 2
    else:
        opcion = 0
    
    resto = valor[4:]
    
    # Regex flexible que permite números como 102, 102., 102.50
    patron = r"(\d+(?:\.\d*)?)([a-zA-Z]+)$"
    match = re.search(patron, resto)
    
    if match:
        strike_raw = match.group(1).rstrip('.')  # elimina el punto si está solo
        try:
            strike = float(strike_raw)
        except:
            strike = None
        vencimiento = match.group(2)
        return pd.Series([opcion, strike, vencimiento])
    else:
        return pd.Series([opcion, None, None])

# Aplicar al DataFrame
df_final[['opcion', 'strike', 'Mes_Vto']] = df_final['Serie'].apply(separar_celda)

# Convertir tipos para optimizar memoria
df_final['opcion'] = df_final['opcion'].astype('int8')
df_final['strike'] = df_final['strike'].astype('float32')


In [134]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108273 entries, 0 to 108272
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Fecha                   108273 non-null  datetime64[ns]
 1   Serie                   108273 non-null  object        
 2   Ultimo_Precio           108273 non-null  float64       
 3   Variacion_%             104306 non-null  float64       
 4   Min                     108273 non-null  float64       
 5   Max                     108273 non-null  float64       
 6   Lotes                   108273 non-null  float64       
 7   Volumen_$               108273 non-null  float64       
 8   Ult_Precio/Cotiz_sub_%  108273 non-null  float64       
 9   Prima_Teorica           108273 non-null  float64       
 10  Delta                   108273 non-null  float64       
 11  Efec_Palanca            99939 non-null   float64       
 12  Volat_Implicita_%       108273

In [136]:
print("Nulos:", df_final['Mes_Vto'].isna().sum())
print("Vacíos o espacios:", (df_final['Mes_Vto'].astype(str).str.strip() == '').sum())


Nulos: 0
Vacíos o espacios: 0


In [137]:
# Reemplazar string en vencimiento por numeros
reemplazos = {
    'DI': 12, 'OC': 10, 'AB': 4, 'FE': 2, 'JU': 6,
    'G': 8, 'AG': 8, 'J': 6, 'O': 10, 'D': 12,
    'F': 2, 'A': 4, 'NO': 11, 'MY': 5, 'MA': 3,
    'JL': 7, 'SE': 9, 'EN': 1, 'Y': 5, 'E': 1
}
df_final['Mes_Vto'] = df_final['Mes_Vto'].replace(reemplazos).astype('int8')



C:\Users\Pablo\AppData\Local\Temp\ipykernel_17036\2107560100.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final['Mes_Vto'] = df_final['Mes_Vto'].replace(reemplazos).astype('int8')


In [138]:
print("Distribución en 'opcion':")
print(df_final['opcion'].value_counts())

print("\nDistribución en 'vencimiento':")
print(df_final['Mes_Vto'].value_counts())


Distribución en 'opcion':
opcion
1    69767
2    38506
Name: count, dtype: int64

Distribución en 'vencimiento':
Mes_Vto
2     18752
4     18697
10    18499
12    18201
6     16648
8     16278
11      282
3       213
9       210
5       204
1       157
7       132
Name: count, dtype: int64


In [141]:
df_final

,Fecha,Serie,Ultimo_Precio,Variacion_%,Min,Max,Lotes,Volumen_$,Ult_Precio/Cotiz_sub_%,Prima_Teorica,Delta,Efec_Palanca,Volat_Implicita_%,Delta_Implic,strike,opcion,Mes_Vto
0,2010-01-04,GFGV2.05EN,0.020,-66.67,0.020,0.020,1000.0,2000.0,0.92,0.008,-0.128,33.50,45.65,-0.199,2.05,2,1
1,2010-01-04,GFGC2.25EN,0.039,11.43,0.039,0.039,200.0,780.0,1.79,0.026,0.321,27.21,42.78,0.364,2.25,1,1
2,2010-01-04,GFGC1.75FE,0.437,1.63,0.437,0.437,30.0,1311.0,20.05,0.453,0.978,4.70,0.00,1.000,1.75,1,2
3,2010-01-04,GFGC1.85FE,0.370,8.82,0.370,0.370,30.0,1110.0,16.97,0.360,0.938,5.69,42.05,0.894,1.85,1,2
4,2010-01-04,GFGC1.95FE,0.290,52.63,0.290,0.300,665.0,19450.0,13.30,0.272,0.864,6.91,42.54,0.813,1.95,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108268,2024-04-23,GFGC4050JU,51.874,-16.47,50.000,67.000,2849.0,17442803.0,1.62,120.850,0.309,8.21,35.89,0.210,4050.00,1,6
108269,2024-04-23,GFGC4200JU,35.189,-20.38,32.990,48.990,6278.0,26059354.0,1.10,92.060,0.251,8.76,36.70,0.152,4200.00,1,6
108270,2024-04-23,GFGC2800AG,800.000,14.29,800.000,800.000,25.0,2000000.0,24.92,975.826,0.904,2.97,0.00,1.000,2800.00,1,8
108271,2024-04-23,GFGC2900JU,584.000,-10.34,580.000,694.000,177.0,10984764.0,18.19,651.873,0.860,4.24,0.00,1.000,2900.00,1,6


In [142]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108273 entries, 0 to 108272
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Fecha                   108273 non-null  datetime64[ns]
 1   Serie                   108273 non-null  object        
 2   Ultimo_Precio           108273 non-null  float64       
 3   Variacion_%             104306 non-null  float64       
 4   Min                     108273 non-null  float64       
 5   Max                     108273 non-null  float64       
 6   Lotes                   108273 non-null  float64       
 7   Volumen_$               108273 non-null  float64       
 8   Ult_Precio/Cotiz_sub_%  108273 non-null  float64       
 9   Prima_Teorica           108273 non-null  float64       
 10  Delta                   108273 non-null  float64       
 11  Efec_Palanca            99939 non-null   float64       
 12  Volat_Implicita_%       108273

In [143]:
# Guardar como parquet:
df_final.to_parquet('df_opciones.parquet', index=False, compression='snappy')